# BasicTick: Create Cluster: GW
This notebook will create (start) a Gateway cluster.

In [1]:
%%html
<style>
table {float:left}
</style>

## Setup

### Node Types
|Type|Mem (GB)|vCPUs|
|:---|---:|---:|
|kx.s.large|12|2|
|kx.s.xlarge|27|4|
|kx.s.2xlarge|54|8|
|kx.s.4xlarge|108|16|
|kx.s.8xlarge|216|32|
|kx.s.16xlarge|432|64|
|kx.s.32xlarge|864|128|


In [2]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env2 import *

# Managed KX Database and Cluster names to create
SEC_THREADS='8'

BASE_NAME="GATEWAY"
CLUSTER_NAME=f"{BASE_NAME}_20230601"

# Cluster Settings
CODEBASE="basictick"
S3_CODE_PATH="code"

CAPACITY_CONFIG={ 'nodeCount': 1, 'nodeType': 'kx.s.4xlarge'}
CODE_CONFIG={ 's3Bucket': S3_BUCKET, 's3Key': f'{S3_CODE_PATH}/{CODEBASE}.zip' }
INIT_SCRIPT=f'{CODEBASE}/gwmkdbcluster.q'

RDB_CLUSTER_NAME="RDB_basictickdb_20230601"
HDB_CLUSTER_NAME="HDB_basictickdb_20230601"

EXECUTION_ROLE="arn:aws:iam::612841383594:role/kdb-cluster-user"

CMD_ARGS=[
    { 'key': 's', 'value': SEC_THREADS }, 
    { 'key': 'codebase', 'value': CODEBASE}, 
    { 'key': 'rdb_name', 'value': RDB_CLUSTER_NAME}, 
    { 'key': 'hdb_name', 'value': HDB_CLUSTER_NAME}, 
]

In [3]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


# Deploy code

In [4]:
if AWS_ACCESS_KEY_ID is not None:
    cp = f"""
export AWS_ACCESS_KEY_ID={AWS_ACCESS_KEY_ID}
export AWS_SECRET_ACCESS_KEY={AWS_SECRET_ACCESS_KEY}
export AWS_SESSION_TOKEN={AWS_SESSION_TOKEN}

zip -r -X {CODEBASE}.zip {CODEBASE}    
aws s3 cp {CODEBASE}.zip s3://{S3_BUCKET}/{S3_CODE_PATH}/{CODEBASE}.zip
aws s3 ls {S3_BUCKET}/{S3_CODE_PATH}/
"""
else:
    cp = f"""
zip -r -X {CODEBASE}.zip {CODEBASE}    
aws s3 cp {CODEBASE}.zip s3://{S3_BUCKET}/{S3_CODE_PATH}/{CODEBASE}.zip
aws s3 ls {S3_BUCKET}/{S3_CODE_PATH}/
"""
    
# execute the S3 copy
os.system(cp)

updating: basictick/ (stored 0%)
updating: basictick/tp.q (deflated 52%)
updating: basictick/query.q (deflated 19%)
updating: basictick/loadDep.q (deflated 57%)
updating: basictick/funcDownHandle.q (deflated 33%)
updating: basictick/example.schema.q (deflated 12%)
updating: basictick/.ipynb_checkpoints/ (stored 0%)
updating: basictick/.ipynb_checkpoints/query-checkpoint.q (deflated 19%)
updating: basictick/.ipynb_checkpoints/tp-checkpoint.q (deflated 52%)
updating: basictick/.ipynb_checkpoints/tickerplant2-checkpoint (stored 0%)
updating: basictick/.ipynb_checkpoints/tickerplant3-checkpoint (stored 0%)
updating: basictick/tickerplant2 (stored 0%)
updating: basictick/tickerplant3 (stored 0%)
updating: basictick/rdbmkdb.q (deflated 57%)
updating: basictick/hdbmkdb.q (deflated 52%)
updating: basictick/gwmkdbcluster.q (deflated 61%)
updating: basictick/feedmkdb.q (deflated 53%)
updating: basictick/connectmkdb.q (deflated 63%)
updating: basictick/.ipynb_checkpoints/rdbmkdb-checkpoint.q (def

0

## Create Cluster

In [5]:
resp = client.create_kx_cluster(
    environmentId=ENV_ID, 
    clusterName=CLUSTER_NAME,
    clusterType='GATEWAY',
    releaseLabel = '1.0',
    capacityConfiguration=CAPACITY_CONFIG,
    clusterDescription="Created with create_GW notebook",
    executionRole=EXECUTION_ROLE,
    code=CODE_CONFIG,
    initializationScript=INIT_SCRIPT,
    commandLineArguments=CMD_ARGS,
    azMode=AZ_MODE,
    availabilityZoneId=AZ_ID,
    vpcConfiguration={ 
        'vpcId': VPC_ID,
        'securityGroupIds': SECURITY_GROUPS,
        'subnetIds': SUBNET_IDS,
        'ipAddressType': 'IP_V4' }
)

In [6]:
resp

{'ResponseMetadata': {'RequestId': 'c2cbfaaf-9d87-404a-9012-4df8fd42cc4f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1076',
   'connection': 'keep-alive',
   'date': 'Thu, 01 Jun 2023 15:04:19 GMT',
   'x-amzn-requestid': 'c2cbfaaf-9d87-404a-9012-4df8fd42cc4f',
   'x-amz-apigw-id': 'F2D5jFZgoAMFe9w=',
   'x-amzn-trace-id': 'Root=1-6478b370-14f7e1957219eef5558ee360',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 68fbda872a4e92e0774a97bdd960d43a.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': 'sa6ccSpQzeSd9eUfjr6hO3ymWUIk-_lj78Gqw3N1fMJ7DaaWgE11zQ=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'GATEWAY_20230601',
 'clusterType': 'GATEWAY',
 'clusterDescription': 'Created with create_GW notebook',
 'capacityConfiguration': {'nodeType': 'kx.s.4xlarge', 'nodeCount': 1},
 'releaseLabel': '1.0',
 'vpcConfiguration': {'vpcId': 'vpc-0e702dec545865b11',
  'securityGroupIds': ['sg-0

In [7]:
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, show_wait=True)
print()
print("** DONE **")

Cluster: GATEWAY_20230601 status is PENDING, total wait 0:00:00, waiting 30 sec ...
Cluster: GATEWAY_20230601 status is CREATING, total wait 0:00:30, waiting 30 sec ...
Cluster: GATEWAY_20230601 status is CREATING, total wait 0:01:00, waiting 30 sec ...
Cluster: GATEWAY_20230601 status is CREATING, total wait 0:01:30, waiting 30 sec ...
Cluster: GATEWAY_20230601 status is CREATING, total wait 0:02:00, waiting 30 sec ...
Cluster: GATEWAY_20230601 status is CREATING, total wait 0:02:30, waiting 30 sec ...
Cluster: GATEWAY_20230601 status is CREATING, total wait 0:03:00, waiting 30 sec ...
Cluster: GATEWAY_20230601 status is CREATING, total wait 0:03:30, waiting 30 sec ...
Cluster: GATEWAY_20230601 status is CREATING, total wait 0:04:00, waiting 30 sec ...
Cluster: GATEWAY_20230601 status is CREATING, total wait 0:04:30, waiting 30 sec ...
Cluster: GATEWAY_20230601 status is CREATING, total wait 0:05:00, waiting 30 sec ...
Cluster: GATEWAY_20230601 status is CREATING, total wait 0:05:30, 

In [8]:
# Give permissions time to propogate after cluster creation....
time.sleep(20)

In [9]:
conn_str = get_kx_connection_string(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, userName=KDB_USERNAME, boto_session=session)
print(conn_str)

:tcps://vpce-0d944f826c07ef733-2vvxlr7i.vpce-svc-0d09e3de8dc07dc10.us-east-1.vpce.amazonaws.com:443:bob:Host=vpce-0d944f826c07ef733-2vvxlr7i.vpce-svc-0d09e3de8dc07dc10.us-east-1.vpce.amazonaws.com&Port=5000&User=bob&Action=finspace%3AConnectKxCluster&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEIj%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJGMEQCIEDckMBN55eQV31Qr7qYow2wcIbMtKJo7JYGSKl99f91AiBieyMRdODI0FTwvpz8pAtdrict3Ax542ZVSvFjYNmkLCqBAwjA%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F8BEAAaDDYxMjg0MTM4MzU5NCIMNbBNJfLQOrQwhqBvKtUCH8XG9y2WQTnVxwWOPKtDo0NPRFqQDWggFlbzUF69c1H4swtTiZ3SPapnugCwH%2Fm4y8zdVcTtXpG1fmtaBQmI4wV9EbVMn%2F5Lky9ynecYSGfJ3K5flP9Y9YTbh2ukpXASEyonP%2BiGdgSRn%2BG2gpbdJzeyIBYX5Di3IJueu80bJ861C3xD%2BSHNOBeWNrTiaSF3al0cwobxFZH83mMEFcRt5%2B98FgilEgwC6xODRRcegoInK18jQlMBbvg%2FYR2T%2BkwUe1F4nxUmMVR7%2FXkO%2B6T8%2FnqVTb6V33JhI66H7fgtKa51bQbNNL4sObFvgqcM8G72vYKjUoUGaRaVaJZby6mn%2B65Cy5Svnz1WzPfsGf%2FygVIFQZlEKgcRgRxNzuzxzE4T1O1TLtmyvOjJdvFKllG3t9Cdn9vz8A38zYtxr%2F9SSrtXSBnd1RMZhqoj1cay8h%2FfsBnvEI

In [10]:
cdf = get_clusters(client, ENV_ID, clusterType="GATEWAY")

display(cdf)

clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName
GATEWAY_20230601,RUNNING,GATEWAY,"{'nodeType': 'kx.s.4xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '8'}, {'key': 'codebase', 'value': 'basictick'}, {'key': 'rdb_name', 'value': 'RDB_basictickdb_20230601'}, {'key': 'hdb_name', 'value': 'HDB_basictickdb_20230601'}]",Created with create_GW notebook,2023-06-01 15:18:14.007000+00:00,2023-06-01 15:04:19.150000+00:00,None
GATEWAY_20230601a,DELETING,GATEWAY,"{'nodeType': 'kx.s.8xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '8'}, {'key': 'codebase', 'value': 'basictick'}, {'key': 'rdb_name', 'value': 'RDB_basictickdb_20230601'}, {'key': 'hdb_name', 'value': 'HDB_basictickdb_20230601'}]",Created with create_GW notebook,2023-06-01 15:06:20.481000+00:00,2023-06-01 14:45:14.851000+00:00,None
GATEWAY_basictickdb_20230601,DELETING,GATEWAY,"{'nodeType': 'kx.s.4xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'basictickdb'}, {'key': 'codebase', 'value': 'basictick'}, {'key': 'rdb_name', 'value': 'RDB_basictickdb_20230601'}, {'key': 'hdb_name', 'value': 'HDB_basictickdb_20230601'}]",Created with create_GW notebook,2023-06-01 15:02:08.380000+00:00,2023-06-01 14:41:40.010000+00:00,None


In [11]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2023-06-01 15:18:56.220549
